## Retrieve Weather Data

In [1]:
# Import libraries.
import pandas as pd
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key

In [2]:
# Select US cities.
lats = np.random.uniform(low=20, high=70, size=7000)
lngs = np.random.uniform(low=-160, high=-65, size=7000)
lat_lng = zip(lats, lngs)

In [3]:
coordinates = list(lat_lng)

In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

1211

In [6]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_des = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Current Description": city_weather_des,
                         "Country": city_country})
        
        # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | chowchilla
Processing Record 2 of Set 1 | ferme-neuve
Processing Record 3 of Set 1 | sitka
Processing Record 4 of Set 1 | fairbanks
Processing Record 5 of Set 1 | norman wells
Processing Record 6 of Set 1 | port hardy
Processing Record 7 of Set 1 | lompoc
Processing Record 8 of Set 1 | marysville
Processing Record 9 of Set 1 | carlyle
Processing Record 10 of Set 1 | stettler
Processing Record 11 of Set 1 | jonesboro
Processing Record 12 of Set 1 | hilo
Processing Record 13 of Set 1 | whitehorse
Processing Record 14 of Set 1 | palmer
Processing Record 15 of Set 1 | linares
Processing Record 16 of Set 1 | athabasca
Processing Record 17 of Set 1 | chapais
Processing Record 18 of Set 1 | guerrero negro
Processing Record 19 of Set 1 | kodiak
Processing Record 20 of Set 1 | georgetown
Processing Record 21 of Set 1 | haines junction
Processing Record 22 of Set 1 | yellowknife
Processing Record 23 of Set

In [7]:
city_df = pd.DataFrame(city_data)
city_df.shape

(1192, 9)

In [8]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", 
                    "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_df = city_df[new_column_order]
city_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Chowchilla,US,37.1230,-120.2602,100.02,29,0,10.36,clear sky
1,Ferme-Neuve,CA,46.7001,-75.4493,60.94,67,90,11.23,overcast clouds
2,Sitka,US,57.0531,-135.3300,57.09,77,100,10.36,overcast clouds
3,Fairbanks,US,64.8378,-147.7164,80.46,43,100,0.00,smoke
4,Norman Wells,CA,65.2820,-126.8329,78.84,44,75,11.50,broken clouds


In [9]:
new_df = city_df[city_df.Country == 'US']
new_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Chowchilla,US,37.1230,-120.2602,100.02,29,0,10.36,clear sky
2,Sitka,US,57.0531,-135.3300,57.09,77,100,10.36,overcast clouds
3,Fairbanks,US,64.8378,-147.7164,80.46,43,100,0.00,smoke
6,Lompoc,US,34.6391,-120.4579,89.74,57,0,13.80,clear sky
7,Marysville,US,48.0518,-122.1771,74.17,78,100,4.61,overcast clouds


In [10]:
new_df.shape

(693, 9)

In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
new_df.to_csv(output_data_file, index_label="City_ID")